<a href="https://colab.research.google.com/github/deondrae4088/WebMd_chat/blob/Project-3-(Gabe's-Branch)/Project3Gabe'sBranch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow kagglehub
import tensorflow as tf
import pathlib
import kagglehub
import os
import zipfile
import glob
import PIL
from collections import defaultdict
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.model_selection import train_test_split

dataset_names = [
    "shubhamgoel27/dermnet",
    "pkdarabi/brain-tumor-image-dataset-semantic-segmentation",
    "paultimothymooney/chest-xray-pneumonia",
    "maedemaftouni/covid19-ct-scan-lesion-segmentation-dataset",
    "aryashah2k/breast-ultrasound-images-dataset"
]

extracted_dirs = [pathlib.Path(os.path.join(os.getcwd(), name.split('/')[1])) if (download_path := kagglehub.dataset_download(name)).endswith(".zip") and (os.makedirs(os.path.join(os.getcwd(), name.split('/')[1]), exist_ok=True) or True) and zipfile.ZipFile(download_path, 'r').extractall(os.path.join(os.getcwd(), name.split('/')[1])) else pathlib.Path(download_path)
                  for name in dataset_names
                  if print(f"Downloaded '{name}' to: {download_path}") or True]

all_images = [p for img_dir in extracted_dirs if print(f"Processing: {img_dir}") or True
              for ext in ['**/*.jpg', '**/*.jpeg', '**/*.png']
              for p in img_dir.glob(ext) if print(f"  Found {len(list(img_dir.glob(ext)))} images") or True]

print(f"\nTotal images found: {len(all_images)}")
all_images and (print(f"First image found: {all_images[0]}") or True) and (lambda p: (print(f"Image size: {PIL.Image.open(str(p)).size}") if (img := PIL.Image.open(str(p))) else None) if print(f"Error opening image: {e}") is None else None)(all_images[0])




Streaming output truncated to the last 5000 lines.
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Found 19559 images
  Fo

In [ ]:
organize_images_by_class = lambda root_dir, image_extensions=['*.jpg', '*.jpeg', '*.png']: defaultdict(list, {
    item.name: [p for ext in image_extensions for p in item.glob(ext)]
    for item in pathlib.Path(root_dir).iterdir() if item.is_dir()
})

all_organized_images = defaultdict(list)
[all_organized_images.update(organize_images_by_class(dataset_dir)) for dataset_dir in extracted_dirs]

print("\nOrganized images by class:") if all_organized_images else print("\nNo images organized by class found.")
[print(f"Class: {cls}, Found {len(imgs)} images") for cls, imgs in all_organized_images.items()]


In [ ]:
if all_organized_images:
    Sequential = tf.keras.models.Sequential
    layers = tf.keras.layers
    img_height = 224
    img_width = 224
    num_classes = len(all_organized_images)

    model = Sequential([
        layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 3, padding='same', activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()

    load_and_preprocess_image = lambda img_path, target_size=(img_height, img_width): image.img_to_array(image.load_img(img_path, target_size=target_size)) / 255.0

    X = np.array([load_and_preprocess_image(str(img_path))
                  for class_name, img_paths in all_organized_images.items()
                  for img_path in img_paths])
    y = np.array([i for i, class_name in enumerate(all_organized_images.keys())
                  for _ in all_organized_images[class_name]])
    y = tf.keras.utils.to_categorical(y, num_classes=num_classes)

    print("\nShape of image data (X):", X.shape)
    print("Shape of labels (y):", y.shape)

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

else:
    print("\nNo organized images found to build the model.")


In [ ]:
# This is a simple Gradio app that uses a *trained* model to classify images
# based on the labels it learned during training.
import gradio as gr
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image

# --- Assume you have a trained model and class labels ---
# Replace these placeholders with your actual trained model and labels

# Example of a simple CNN model (replace with your actual model)
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc = nn.Linear(32 * 56 * 56, num_classes) # Adjust input size based on your training image size

    def forward(self, x):
        x = self.maxpool(self.relu(self.conv1(x)))
        x = self.maxpool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 56 * 56) # Flatten
        x = self.fc(x)
        return x

# Assume you have the number of classes from your training data
num_classes = 10  # Replace with the actual number of classes

# Load your trained model
model = SimpleCNN(num_classes)
# Replace 'path/to/your/trained_model.pth' with the actual path to your saved model
try:
    model.load_state_dict(torch.load('path/to/your/trained_model.pth'))
    model.eval()  # Set the model to evaluation mode
    print("Trained model loaded successfully.")
except FileNotFoundError:
    print("Error: Trained model file not found. Please provide the correct path.")
    exit()

# Assume you have the class labels corresponding to the model's output
# Replace this with your actual list of class labels
class_labels = ['class_0', 'class_1', 'class_2', 'class_3', 'class_4',
                'class_5', 'class_6', 'class_7', 'class_8', 'class_9']

# Define the transformations used during training
# Make sure these are the *same* transformations you applied to your training data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Or the size you trained on
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Or your training normalization
])

def classify_image(img):
    """Classifies the input image using the loaded trained model."""
    try:
        img = Image.fromarray(img).convert('RGB') # Ensure image is RGB
        img_t = transform(img).unsqueeze(0)

        with torch.no_grad():
            out = model(img_t)
            probabilities = F.softmax(out, dim=1)
            _, predicted_idx = torch.max(probabilities, 1)
            predicted_label = class_labels[predicted_idx.item()]
            confidence = probabilities[0, predicted_idx.item()].item()

        return f"Predicted Class: {predicted_label} (Confidence: {confidence:.4f})"

    except Exception as e:
        return f"Error during classification: {e}"

demo = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="numpy"),
    outputs="text",
    title="Image Classification with Trained Model",
    description="Upload an image to classify it based on the model it was trained on."
)

demo.launch()
)

In [ ]:
## Create a Gradio interface for the image classification model
demo= gr.Interface(
    fn=lambda img: img. inputs= "image"
    outputs="text",
    title="Image Classification Demo", )
launch(show_error=True)
app.launch(show_error=True)

In [ ]:
## Define a custom theme for the Gradio app
gr.themes.builder()
class Seafoam (Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.background_color = "#008080"
        self.text_color = "#000000"
        self.button_color = "#800020"
        self.button_text_color = "#000000"
        self.input_background_color = "#FFFFFF"
        self.input_text_color = "#000000"
        self.output_background_color = "#FFFFFF"
        self.output_text_color = "#000000"
#   Create a Gradio Blocks app with the custom theme
    with gr.Blocks(theme=Seafoam(primary_hue="008080", secondary_hue="800020")) as demo:
        gr.Markdown("# Image Classification with ResNet50")
        gr.Markdown("Upload an image to classify it.")
        gr.Markdown ("ask a medical question")
        image_input = gr.Image(type="pil", label="Input Image")
        image_classify_button = gr.Button("Classify Image")
        output_text = gr.Textbox(label="Classification Result")
        textbox = gr.Textbox(label="Ask a medical question")
        text_classify_button = gr.Button("Ask")
        text_classify_button.click(predict_fn, inputs=image_input, outputs=output_text)
        text_classify_button.click(predict_fn, inputs=textbox, outputs=output_text)
# Set spacing and radius sizes for the Blocks app
with gr.Blocks(theme=gr.themes.Default(spacing_size="md", radius_size="md")) as demo:
# Set fonts for the Blocks app
with gr.Blocks(theme=gr.themes.Default(font=[gr.themes.GoogleFont("Helvetica"), "Arial", "sans-serif"])) as demo: